In [1]:
import pandas as pd
import mysql.connector
import re
conn = mysql.connector.connect(username= "root",
                               host = "127.0.0.1",
                               password = "Admin@123",
                               database = "dataspark")

cursor = conn.cursor()

In [2]:
df_sales = pd.read_csv('Sales.csv')

In [3]:
df_sales

,Order Number,Line Item,Order Date,Delivery Date,CustomerKey,StoreKey,ProductKey,Quantity,Currency Code
0,366000,1,1/1/2016,NaN,265598,10,1304,1,CAD
1,366001,1,1/1/2016,1/13/2016,1269051,0,1048,2,USD
2,366001,2,1/1/2016,1/13/2016,1269051,0,2007,1,USD
3,366002,1,1/1/2016,1/12/2016,266019,0,1106,7,CAD
4,366002,2,1/1/2016,1/12/2016,266019,0,373,1,CAD
...,...,...,...,...,...,...,...,...,...
62879,2243030,1,2/20/2021,NaN,1216913,43,632,3,USD
62880,2243031,1,2/20/2021,2/24/2021,511229,0,98,4,EUR
62881,2243032,1,2/20/2021,2/23/2021,331277,0,1613,2,CAD
62882,2243032,2,2/20/2021,2/23/2021,331277,0,1717,2,CAD


In [4]:
df_sales.shape

(62884, 9)

In [10]:
df_sales.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 62884 entries, 0 to 62883
Data columns (total 9 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   Order Number   62884 non-null  int64 
 1   Line Item      62884 non-null  int64 
 2   Order Date     62884 non-null  object
 3   Delivery Date  13165 non-null  object
 4   CustomerKey    62884 non-null  int64 
 5   StoreKey       62884 non-null  int64 
 6   ProductKey     62884 non-null  int64 
 7   Quantity       62884 non-null  int64 
 8   Currency Code  62884 non-null  object
dtypes: int64(6), object(3)
memory usage: 4.3+ MB


In [5]:
df_sales.describe()

,Order Number,Line Item,CustomerKey,StoreKey,ProductKey,Quantity
count,6.288400e+04,62884.000000,6.288400e+04,62884.000000,62884.000000,62884.000000
mean,1.430905e+06,2.164207,1.180797e+06,31.802144,1125.859344,3.144790
std,4.532963e+05,1.365170,5.859634e+05,22.978188,709.244010,2.256371
min,3.660000e+05,1.000000,3.010000e+02,0.000000,1.000000,1.000000
25%,1.121017e+06,1.000000,6.808580e+05,8.000000,437.000000,1.000000
50%,1.498016e+06,2.000000,1.261200e+06,37.000000,1358.000000,2.000000
75%,1.788010e+06,3.000000,1.686496e+06,53.000000,1650.000000,4.000000
max,2.243032e+06,7.000000,2.099937e+06,66.000000,2517.000000,10.000000


In [6]:
df_sales.isna().sum()

Order Number         0
Line Item            0
Order Date           0
Delivery Date    49719
CustomerKey          0
StoreKey             0
ProductKey           0
Quantity             0
Currency Code        0
dtype: int64

In [7]:
df_sales.columns = df_sales.columns.str.lower()
df_sales.columns = df_sales.columns.str.replace(" ","_")
df_sales.columns

Index(['order_number', 'line_item', 'order_date', 'delivery_date',
       'customerkey', 'storekey', 'productkey', 'quantity', 'currency_code'],
      dtype='object')

In [8]:
df_sales.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 62884 entries, 0 to 62883
Data columns (total 9 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   order_number   62884 non-null  int64 
 1   line_item      62884 non-null  int64 
 2   order_date     62884 non-null  object
 3   delivery_date  13165 non-null  object
 4   customerkey    62884 non-null  int64 
 5   storekey       62884 non-null  int64 
 6   productkey     62884 non-null  int64 
 7   quantity       62884 non-null  int64 
 8   currency_code  62884 non-null  object
dtypes: int64(6), object(3)
memory usage: 4.3+ MB


In [9]:
df_sales['order_date']= pd.to_datetime(df_sales['order_date'])
df_sales['delivery_date']= pd.to_datetime(df_sales['delivery_date'])

In [10]:
df_sales["shipping_days"] = (df_sales['delivery_date'] - df_sales['order_date']).dt.days

In [31]:
df_sales.head()

,order_number,line_item,order_date,delivery_date,customerkey,storekey,productkey,quantity,currency_code,shipping_days
0,366000,1,2016-01-01,2016-01-05,265598,10,1304,1,CAD,4
1,366001,1,2016-01-01,2016-01-13,1269051,0,1048,2,USD,12
2,366001,2,2016-01-01,2016-01-13,1269051,0,2007,1,USD,12
3,366002,1,2016-01-01,2016-01-12,266019,0,1106,7,CAD,11
4,366002,2,2016-01-01,2016-01-12,266019,0,373,1,CAD,11


In [11]:
df_sales.describe()

,order_number,line_item,order_date,delivery_date,customerkey,storekey,productkey,quantity,shipping_days
count,6.288400e+04,62884.000000,62884,13165,6.288400e+04,62884.000000,62884.000000,62884.000000,13165.000000
mean,1.430905e+06,2.164207,2018-11-30 21:21:56.939126016,2019-01-16 16:14:21.921762304,1.180797e+06,31.802144,1125.859344,3.144790,4.529130
min,3.660000e+05,1.000000,2016-01-01 00:00:00,2016-01-06 00:00:00,3.010000e+02,0.000000,1.000000,1.000000,1.000000
25%,1.121017e+06,1.000000,2018-01-25 00:00:00,2018-05-05 00:00:00,6.808580e+05,8.000000,437.000000,1.000000,3.000000
50%,1.498016e+06,2.000000,2019-02-06 00:00:00,2019-03-05 00:00:00,1.261200e+06,37.000000,1358.000000,2.000000,4.000000
75%,1.788010e+06,3.000000,2019-11-23 00:00:00,2019-12-18 00:00:00,1.686496e+06,53.000000,1650.000000,4.000000,6.000000
max,2.243032e+06,7.000000,2021-02-20 00:00:00,2021-02-27 00:00:00,2.099937e+06,66.000000,2517.000000,10.000000,17.000000
std,4.532963e+05,1.365170,NaN,NaN,5.859634e+05,22.978188,709.244010,2.256371,2.121071


In [12]:
df_sales['delivery_date'] = df_sales['delivery_date'].fillna((df_sales['order_date'] + pd.Timedelta(days=4)))

In [16]:
df_sales["shipping_days"] = (df_sales['delivery_date'] - df_sales['order_date']).dt.days

In [17]:
df_sales.head()

,order_number,line_item,order_date,delivery_date,customerkey,storekey,productkey,quantity,currency_code,shipping_days
0,366000,1,2016-01-01,2016-01-05,265598,10,1304,1,CAD,4
1,366001,1,2016-01-01,2016-01-13,1269051,0,1048,2,USD,12
2,366001,2,2016-01-01,2016-01-13,1269051,0,2007,1,USD,12
3,366002,1,2016-01-01,2016-01-12,266019,0,1106,7,CAD,11
4,366002,2,2016-01-01,2016-01-12,266019,0,373,1,CAD,11


In [18]:
df_sales.isna().sum()

order_number     0
line_item        0
order_date       0
delivery_date    0
customerkey      0
storekey         0
productkey       0
quantity         0
currency_code    0
shipping_days    0
dtype: int64

In [19]:
insert_sales_values = '''
    INSERT INTO sales (
        order_number, 
        line_item, 
        order_date, 
        delivery_date, 
        customerkey, 
        storekey, 
        productkey, 
        quantity, 
        currency_code, 
        shipping_days
    ) VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
'''

# Insert data into the sales table
for index, row in df_sales.iterrows():
    cursor.execute(insert_sales_values, (
        row['order_number'],
        row['line_item'],
        row['order_date'],
        row['delivery_date'],
        row['customerkey'],
        row['storekey'],
        row['productkey'],
        row['quantity'],
        row['currency_code'],
        row['shipping_days']
    ))

conn.commit()    